# The feedforward neural network model for learning PageRank

In [1]:
import time
from mygraph import MyGraph
from helpers import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [3]:
# Pageank as the target or label data
target_pagerank = np.load("fb_co_pages_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([9.98507331e-05])

In [4]:
# scaling the PageRank data
target_pagerank = target_pagerank * 10000
target_pagerank[0]

array([0.99850733])

In [5]:
target_pagerank.shape

(14113, 1)

In [6]:
# the order 5 NFDC matrix as the feature set
matrix_1 = np.load("fb_co_pages_NDFC_matrix_r1-30_sta1_max50_rad5.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [7]:
matrix_1.shape

(14113, 6, 17)

In [8]:
matrix_1[111]

array([[0.      , 0.      , 0.      , 2.      , 1.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 1.      , 1.666667, 0.333333, 0.      ,
        1.      , 0.333333, 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.666667, 0.      , 0.      ],
       [0.6     , 0.2     , 0.8     , 4.      , 4.6     , 5.8     ,
        5.      , 5.4     , 3.      , 5.      , 2.      , 1.4     ,
        0.8     , 0.8     , 0.4     , 0.2     , 0.4     ],
       [0.319149, 0.590426, 0.579787, 1.5     , 1.62766 , 1.867021,
        2.441489, 2.702128, 2.468085, 3.303191, 1.457447, 1.218085,
        0.654255, 0.441489, 1.31383 , 0.265957, 0.308511],
       [0.285945, 0.420032, 0.516424, 1.162089, 1.252558, 1.739365,
        1.776521, 2.116855, 1.785137, 1.413032, 0.898761, 0.796984,
        0.458805, 0.28433 , 0.172321, 0.229402, 0.140011],
       [0.16254 , 0.27820

In [9]:
matrix_1 = matrix_1.reshape(14113,-1)
matrix_1.shape

(14113, 102)

# Converting the data into pytorch tenors

In [10]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([14113, 102]), torch.Size([14113, 1]))

In [11]:
# Activating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[0.9985],
        [0.8875],
        [0.5463],
        ...,
        [0.2811],
        [0.4245],
        [0.3886]], dtype=torch.float64, requires_grad=True)

In [12]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 4113
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([10000, 102]),
 torch.Size([4113, 102]),
 torch.Size([10000, 1]),
 torch.Size([4113, 1]))

## A function for dividing train data into batches

In [13]:
# dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # Dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [14]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

In [15]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (fc1): Linear(in_features=102, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [16]:
# Number of parameters
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print("----------------------")
print(f'Number of all parameters: \n{num_para}')

40800
400
320000
800
160000
200
12800
64
512
8
8
1
----------------------
Number of all parameters: 
535593


In [17]:
# Objective function and optimmizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the Training Loop

In [18]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    TEST LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [19]:
losses = training_loop(n_epochs=2000,
                  batch_size=400,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   0.65423311949,    TEST LOSS:   0.47081270814
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.08209330589,    TEST LOSS:   0.22129108012
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.04253416508,    TEST LOSS:   0.16900217533
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.02602081746,    TEST LOSS:   0.14253042638
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.01821075752,    TEST LOSS:   0.12626607716
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.01198110916,    TEST LOSS:   0.11545421183
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.00921595842,    TEST LOSS:   0.10730690509
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.00920750387,    TEST LOSS:   0.10146991909
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [20]:
# Saving the train losses of pagerank as a numpy array into a file
np.save("ffnn_pagerank_train_losses.npy", losses[0], allow_pickle=False, fix_imports=True)

In [21]:
# Saving the test losses of pagerank as a numpy array into a file
np.save("ffnn_pagerank_test_losses.npy", losses[1], allow_pickle=False, fix_imports=True)

In [22]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,102)
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%100 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------------------------------------")

target 0.8270967186912557,    prediction: 0.8579225540161133
index 0:       inaccuracy: 3.727%
-----------------------------------------------------------------
target 0.476315524567235,    prediction: 0.3173266351222992
index 100:       inaccuracy: 33.379%
-----------------------------------------------------------------
target 0.445962647242291,    prediction: 0.509641706943512
index 200:       inaccuracy: 14.279%
-----------------------------------------------------------------
target 0.36575762176223975,    prediction: 0.45871588587760925
index 300:       inaccuracy: 25.415%
-----------------------------------------------------------------
target 0.4006758708746178,    prediction: 0.39544352889060974
index 400:       inaccuracy: 1.306%
-----------------------------------------------------------------
target 0.6457893514893857,    prediction: 0.665459930896759
index 500:       inaccuracy: 3.046%
-----------------------------------------------------------------
target 0.5121315814962

In [23]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  9.651


In [24]:
torch.save(model, "trained_FFNN_model_pagerank.pt")